In [1]:
import jqdatasdk as jq
import time
from utils.my_cache import cache_wrapper
import pandas as pd
import math
from functools import reduce
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
jq.auth('13764432461', 'Swisschina6')


auth success 


## **I. Data Source Comparison**

most data come from `JoinQuant`

active or passive classification comes from `Win.d`

In [2]:
@cache_wrapper(expire = 60 * 60 * 24 * 7)
def brief_of_funds(frequency : str = time.strftime('%W')) -> pd.DataFrame:
    return jq.get_all_securities(['fund', 'open_fund'], time.strftime('%Y%m%d'))


@cache_wrapper(expire = 60 * 60 * 24 * 7)
def main_info(fund_codes, frequency : str = time.strftime('%W')) -> pd.DataFrame:
    '''
    modify function should be done with deleting cache
    '''
    limit = jq.DBTable.RESULT_ROWS_LIMIT
    fund_code_cuts = np.array_split(fund_codes, math.ceil(len(fund_codes) / limit))
    full_table_cuts = []
    for code_cut in fund_code_cuts:
        # NOTE `in` is not applicable in filter, use `in_()` defined by joinquant
        table_cut = jq.finance.run_query(jq.query(jq.finance.FUND_MAIN_INFO).filter(jq.finance.FUND_MAIN_INFO.main_code.in_(code_cut)))
        full_table_cuts.append(table_cut)
    full_table = pd.concat(full_table_cuts).reset_index(drop=True)
    full_table['start_date'] = full_table.start_date.astype(np.datetime64).dt.to_period('M')
    return full_table

@cache_wrapper(expire = 60 * 60 * 24 * 7)
def portfolio_info(fund_codes, frequency : str = time.strftime('%W')) -> pd.DataFrame:
    limit = jq.DBTable.RESULT_ROWS_LIMIT
    fund_code_cuts = np.array_split(fund_codes, math.ceil(len(fund_codes) * 6 / limit))
    full_table_cuts = []
    for code_cut in fund_code_cuts:
        # NOTE `in` is not applicable in filter, use `in_()` defined by joinquant
        table_cut = jq.finance.run_query(jq.query(jq.finance.FUND_PORTFOLIO).filter(jq.finance.FUND_PORTFOLIO.code.in_(code_cut)))
        full_table_cuts.append(table_cut)
    full_table = pd.concat(full_table_cuts).reset_index(drop=True)
    return full_table



In [8]:
brief = brief_of_funds()
fund_codes = brief.index.str.split('.').str[0]
fund_main_info = main_info(fund_codes)
fund_portfolio_info = portfolio_info(fund_codes)

In [9]:
def round_(data : float):
    return str(round(data, 4)*100) + '%'

maininfo = pd.read_csv('data/FUND_MainInfo.csv', parse_dates=['InceptionDate'])
maininfo['InceptionDate'] = maininfo.InceptionDate.dt.to_period('M')


unitclassinfo = pd.read_csv('data/FUND_UnitClassInfo.csv', parse_dates=['ListingDate', 'InceptionDate'])
unitclassinfo['InceptionDate'] = unitclassinfo.InceptionDate.dt.to_period('M')
unitclassinfo['MasterFundCode'] = unitclassinfo.MasterFundCode

unitclassinfo_ = unitclassinfo.drop_duplicates(subset=['Symbol'], keep='last')
unitclassinfo_ = unitclassinfo_[pd.notnull(unitclassinfo_.MasterFundCode)]
unitclassinfo__ = unitclassinfo_.set_index(['MasterFundCode', 'Symbol'])
unitclassinfo__

selected_columns = ['FundID', 'MasterFundCode', 'FullName', 'Symbol', 'InceptionDate_y', 'Category', 'IsETF', 'IsQDII', 'IsActiveOrPassive']
maininfo_ = pd.merge(maininfo, unitclassinfo_, how='inner' ,on='MasterFundCode')[selected_columns].drop_duplicates(subset='Symbol').reset_index(drop=True)
maininfo_['InceptionDate'] = maininfo_.InceptionDate_y
maininfo_ = maininfo_.drop('InceptionDate_y', axis=1)

In [10]:
maininfo_['Symbol'] = maininfo_.Symbol.astype(str).str.zfill(6)
maininfo_['Category'] = maininfo_['Category'].str.replace('基金', '')
maininfo_ = maininfo_.rename(columns={'Symbol': 'main_code', 'Category': 'csmar_category', 'IsETF': 'csmar_IsETF', 'IsQDII': 'csmar_IsQDII', 'IsActiveOrPassive': 'csmar_IsActiveOrPassive'})

In [11]:
wind_category = pd.read_excel('wind_category.xlsx')
wind_category['证券代码'] = wind_category.证券代码.astype(str).str.split('.').str[0].str.zfill(6)
wind_category = wind_category.rename(columns={'证券代码': 'main_code'})
wind_category['投资类型(一级分类)'] = wind_category['投资类型(一级分类)'].str.replace('基金', '').str.replace('货币市场型', '货币型')
wind_category['投资类型(二级分类)'] = wind_category['投资类型(二级分类)'].apply(lambda style: 2 if '被动' in style else 1)

wind_category = wind_category.rename(columns={'投资类型(一级分类)': 'wind_category', '投资类型(二级分类)': 'wind_active_or_passive'})


In [12]:
temp_merge = pd.merge(fund_main_info, maininfo_, on='main_code')\
    [['name', 'main_code', 'underlying_asset_type', 'csmar_category', 'csmar_IsETF', 'csmar_IsQDII', 'operate_mode', 'csmar_IsActiveOrPassive']]
full_merge = pd.merge(temp_merge, wind_category, on='main_code')\
    [['name', 'main_code', 'underlying_asset_type', 'csmar_category', 'wind_category', 'csmar_IsETF', 'csmar_IsQDII', 'operate_mode', 'wind_active_or_passive', 'csmar_IsActiveOrPassive']]
full_merge

,name,main_code,underlying_asset_type,csmar_category,wind_category,csmar_IsETF,csmar_IsQDII,operate_mode,wind_active_or_passive,csmar_IsActiveOrPassive
0,中海可转债债券,000003,债券型,债券型,债券型,2,2,开放式基金,1,NaN
1,中海可转债C,000004,债券型,债券型,债券型,2,2,开放式基金,1,NaN
2,嘉实增强信用,000005,债券型,债券型,债券型,2,2,开放式基金,1,NaN
3,西部利得量化成长混合,000006,混合型,混合型,混合型,2,2,开放式基金,1,1.0
4,嘉实中证500ETF联接,000008,基金型,股票型,股票型,2,2,开放式基金,2,2.0
...,...,...,...,...,...,...,...,...,...,...
16074,博时特许R,960026,混合型,混合型,混合型,2,2,开放式基金,1,1.0
16075,博时信用R,960027,债券型,债券型,债券型,2,2,开放式基金,1,NaN
16076,建信优选H,960028,混合型,混合型,混合型,2,2,开放式基金,1,1.0
16077,建信双息H,960029,债券型,债券型,债券型,2,2,开放式基金,1,NaN


In [13]:
# NOTE improve: random sample to verify
full_merge[full_merge[['underlying_asset_type', 'csmar_category', 'wind_category']].T.nunique() != 1].head(20)

,name,main_code,underlying_asset_type,csmar_category,wind_category,csmar_IsETF,csmar_IsQDII,operate_mode,wind_active_or_passive,csmar_IsActiveOrPassive
4,嘉实中证500ETF联接,000008,基金型,股票型,股票型,2,2,开放式基金,2,2.0
25,华夏全球,000041,股票型,股票型,国际(QDII),2,1,QDII,1,1.0
27,嘉实美国成长股票（QDII）,000043,股票型,股票型,国际(QDII),2,1,QDII,1,1.0
28,嘉实美股(美元现汇),000044,股票型,股票型,国际(QDII),2,1,QDII,1,1.0
33,中银标普全球资源等权重指数（QDII）,000049,股票型,股票型,国际(QDII),2,1,QDII,1,2.0
34,华夏沪深300ETF联接,000051,基金型,股票型,股票型,2,2,开放式基金,2,2.0
37,广发纳指100ETF联接美元(QDII)A,000055,基金型,股票型,国际(QDII),2,1,QDII,1,2.0
51,华夏恒生ETF联接,000071,基金型,股票型,国际(QDII),2,2,QDII,1,2.0
55,华夏恒生ETF联接A(美元现汇),000075,基金型,股票型,国际(QDII),2,2,QDII,1,2.0
56,华夏恒生ETF联接A(美元现钞),000076,基金型,股票型,国际(QDII),2,2,QDII,1,2.0


<img src="attachment\000008.jpg" width=1200> <img src="attachment\000051.jpg" width=1200> <img src="attachment\000248.jpg" width=1200> <img src="attachment\000071.jpg" width=1200> <img src="attachment\000055.jpg" width=600>


In [14]:
np.setdiff1d(fund_main_info.main_code, wind_category.main_code)[:5]

array(['001620', '001683', '001769', '001772', '001773'], dtype=object)

In [15]:
fund_main_info[fund_main_info.main_code == '001620']

,id,main_code,statistics_main_code,name,advisor,trustee,operate_mode_id,operate_mode,underlying_asset_type_id,underlying_asset_type,invest_style_id,invest_style,start_date,end_date
1178,1049,001620,None,嘉实新机遇,嘉实基金管理有限公司,中国工商银行股份有限公司,401001,开放式基金,402004,混合型,005005,偏股混合型,2015-07,None


<img src="attachment/001620.jpg" width=500> <img src="attachment/001620_.jpg" width=700> <img src="attachment/001620__.jpg" width=700>

In [16]:
final_main_info = pd.merge(fund_main_info, wind_category, on='main_code')[['main_code', 'name', 'operate_mode', 'underlying_asset_type', 'wind_active_or_passive', 'start_date']]
final_main_info

,main_code,name,operate_mode,underlying_asset_type,wind_active_or_passive,start_date
0,000001,华夏成长,开放式基金,混合型,1,2001-12
1,000003,中海可转债债券,开放式基金,债券型,1,2013-03
2,000004,中海可转债C,开放式基金,债券型,1,2013-03
3,000005,嘉实增强信用,开放式基金,债券型,1,2013-03
4,000006,西部利得量化成长混合,开放式基金,混合型,1,2019-03
...,...,...,...,...,...,...
16291,960026,博时特许R,开放式基金,混合型,1,2016-01
16292,960027,博时信用R,开放式基金,债券型,1,2016-01
16293,960028,建信优选H,开放式基金,混合型,1,2016-04
16294,960029,建信双息H,开放式基金,债券型,1,2016-04


## **II. Preprocessing**

In [17]:
specs = [[{'type':'domain'}] * 3]

fig = make_subplots(rows=1, cols=3, specs=specs)

fig.add_trace(go.Pie(labels=final_main_info.underlying_asset_type.value_counts().index, values=final_main_info.underlying_asset_type.value_counts().values), 1, 1)
fig.add_trace(go.Pie(labels=final_main_info.operate_mode.value_counts().index, values=final_main_info.operate_mode.value_counts().values), 1, 2)
fig.add_trace(go.Pie(labels=final_main_info.wind_active_or_passive.replace([1, 2], ['1非被动', '2被动']).value_counts().index, values=final_main_info.wind_active_or_passive.value_counts().values), 1, 3)

fig.update_layout(width=1200, height=500)

### 1. first 4 : static filtering

（1）基金类别必须为“股票型基金”或“混合型基金”；

（2）非 ETF 类基金；

（3）非 QDII 类基金；

（4）主动管理标识为 1 或者为空；

In [18]:
filter_1 = np.logical_or(final_main_info.underlying_asset_type == '股票型', final_main_info.underlying_asset_type == '混合型')
filter_2 = final_main_info.operate_mode != 'ETF'
filter_3 = final_main_info.operate_mode != 'QDII'
filter_4 = final_main_info.wind_active_or_passive != 2

filters = [(1, filter_1), (2, filter_2), (3, filter_3), (4, filter_4)]
np.array(list(map(lambda _: _[1].sum(), filters))) / len(final_main_info)

# proportional data left

array([0.62045901, 0.95747423, 0.97416544, 0.8718704 ])

In [19]:
def func(previous, now):
    if type(previous) == tuple:
        print(f'filter {previous[0]}: {len(final_main_info)} to {previous[1].sum()}, ' + round_(1 - previous[1].sum() / len(final_main_info)) + ' funds dropped')
        temp = np.logical_and(previous[1], now[1])
        print(f'filter {now[0]}: {previous[1].sum()} to {temp.sum()}, ' + round_(1 - temp.sum() / previous[1].sum()) + ' funds dropped')
        return temp
    else:
        temp = np.logical_and(previous, now[1])
        print(f'filter {now[0]}: {previous.sum()} to {temp.sum()}, ' + round_(1 - temp.sum() / previous.sum()) + ' funds dropped')
        return temp


filters_ = reduce(func, filters)

filter 1: 16296 to 10111, 37.95% funds dropped
filter 2: 10111 to 9482, 6.22% funds dropped
filter 3: 9482 to 9202, 2.9499999999999997% funds dropped
filter 4: 9202 to 8643, 6.069999999999999% funds dropped


In [20]:
final_main_info_ = final_main_info[filters_]
final_main_info_.set_index('main_code').sort_index()

,name,operate_mode,underlying_asset_type,wind_active_or_passive,start_date
main_code,,,,,
000001,华夏成长,开放式基金,混合型,1,2001-12
000006,西部利得量化成长混合,开放式基金,混合型,1,2019-03
000011,华夏大盘,开放式基金,混合型,1,2004-08
000017,财通可持续混合,开放式基金,混合型,1,2013-03
000020,景顺品质,开放式基金,混合型,1,2013-03
...,...,...,...,...,...
960023,工银稳健成长混合H,开放式基金,混合型,1,2015-08
960024,嘉实成长H,开放式基金,混合型,1,2016-01
960026,博时特许R,开放式基金,混合型,1,2016-01


### ii. use filtered data (first 4) to create big table

big table: from `fund code` indexed only to `fund code-month` multiIndex

In [21]:
indexes = pd.MultiIndex.from_product([final_main_info_.main_code.sort_values().values, pd.date_range('2002-1', '2022-10', freq='M')])
temp_codes = indexes.get_level_values(0)
temp_date = indexes.get_level_values(1)
main_data = pd.DataFrame(index=indexes)
main_data['main_code'] = temp_codes
main_data['date'] = temp_date.to_period('M')
main_data = pd.merge(main_data, final_main_info_, on='main_code')
main_data = main_data[main_data.date >= main_data.start_date].reset_index(drop=True)
main_data.set_index(['main_code', 'date'])

name operate_mode underlying_asset_type  \
main_code date                                                
000001    2002-01   华夏成长        开放式基金                   混合型   
          2002-02   华夏成长        开放式基金                   混合型   
          2002-03   华夏成长        开放式基金                   混合型   
          2002-04   华夏成长        开放式基金                   混合型   
          2002-05   华夏成长        开放式基金                   混合型   
...                  ...          ...                   ...   
960033    2022-05  农银消费H        开放式基金                   混合型   
          2022-06  农银消费H        开放式基金                   混合型   
          2022-07  农银消费H        开放式基金                   混合型   
          2022-08  农银消费H        开放式基金                   混合型   
          2022-09  农银消费H        开放式基金                   混合型   

                   wind_active_or_passive start_date  
main_code date                                        
000001    2002-01                       1    2001-12  
          2002-02                       1    2001-12  
          2002-03                       1    2001-12  
          2002-04                       1    2001-12  
          2002-05                       1    2001-12  
...                                   ...        ...  
960033    2022-05                       1    2016-07  
          2022-06                       1    2016-07  
          2022-07                       1    2016-07  
          2022-08                       1    2016-07  
          2022-09                       1    2016-07  

[382854 rows x 5 columns]

In [23]:
main_data

,main_code,date,name,operate_mode,underlying_asset_type,wind_active_or_passive,start_date
0,000001,2002-01,华夏成长,开放式基金,混合型,1,2001-12
1,000001,2002-02,华夏成长,开放式基金,混合型,1,2001-12
2,000001,2002-03,华夏成长,开放式基金,混合型,1,2001-12
3,000001,2002-04,华夏成长,开放式基金,混合型,1,2001-12
4,000001,2002-05,华夏成长,开放式基金,混合型,1,2001-12
...,...,...,...,...,...,...,...
382849,960033,2022-05,农银消费H,开放式基金,混合型,1,2016-07
382850,960033,2022-06,农银消费H,开放式基金,混合型,1,2016-07
382851,960033,2022-07,农银消费H,开放式基金,混合型,1,2016-07
382852,960033,2022-08,农银消费H,开放式基金,混合型,1,2016-07


In [24]:
final_main_info_

,main_code,name,operate_mode,underlying_asset_type,wind_active_or_passive,start_date
0,000001,华夏成长,开放式基金,混合型,1,2001-12
4,000006,西部利得量化成长混合,开放式基金,混合型,1,2019-03
8,000011,华夏大盘,开放式基金,混合型,1,2004-08
13,000017,财通可持续混合,开放式基金,混合型,1,2013-03
14,000020,景顺品质,开放式基金,混合型,1,2013-03
...,...,...,...,...,...,...
16289,960023,工银稳健成长混合H,开放式基金,混合型,1,2015-08
16290,960024,嘉实成长H,开放式基金,混合型,1,2016-01
16291,960026,博时特许R,开放式基金,混合型,1,2016-01
16293,960028,建信优选H,开放式基金,混合型,1,2016-04
